In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt
import matplotlib.style as style
import time

In [2]:
style.use('fivethirtyeight')
imputed = 'PATIENT_DATA_ALL_3.csv'
therapy = 'PATIENT_DATA_THERAPY.csv'
merged = pd.read_csv(imputed,sep='\t')
therapy = pd.read_csv(therapy,sep='\t')
cna = pd.read_csv('data_CNA.xls.txt',sep='\t')

In [3]:
cna.set_index('Hugo_Symbol', inplace=True)
cna.drop(['Entrez_Gene_Id'], axis=1,inplace=True)
cna = cna.transpose()
cna = cna[:-1] # getting rid of the last row
cna.sort_index(inplace=True)

merged.sort_values('SAMPLE_ID', ascending=False)
merged.set_index('SAMPLE_ID',inplace=True)
merged.drop('PATIENT_ID', axis=1,inplace=True)
merged.drop(['Unnamed: 0'], axis=1,inplace=True)

therapy.drop('PATIENT_ID', axis=1,inplace=True)
therapy.drop(['Unnamed: 0'], axis=1,inplace=True)
therapy.set_index('SAMPLE_ID', inplace=True)

In [5]:
cna

Hugo_Symbol,DDX11L1,WASH7P,hsa-mir-1302-2,FAM138A,FAM138F,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,...,RPL23AP82,SBF1,SCO2,SELO,SHANK3,SYCE3,TRABD,TUBGCP6,TYMP,ZBED4
MBC-MBCProject_1ps2hZH8-Tumor-SM-DL4OO,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MBC-MBCProject_1qhlIasw-Tumor-SM-AXGNH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MBC-MBCProject_27uAugT4-Tumor-SM-DL45T,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MBC-MBCProject_3LI1TNs3-Tumor-SM-DL43B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MBC-MBCProject_3MUnT7Ty-Tumor-SM-CGLLE,0,0,0,0,0,0,0,0,0,0,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
MBC-MBCProject_43UpSwhz-Tumor-SM-DL4W8,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MBC-MBCProject_4DIpSBFV-Tumor-SM-CGM2Q,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
MBC-MBCProject_4MF1FlFQ-Tumor-SM-AZ5CU,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MBC-MBCProject_4MF1FlFQ-Tumor-SM-CGM4M,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MBC-MBCProject_4OIOfAt9-Tumor-SM-CGLYL,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [116]:
columns = list(therapy.columns)

In [69]:
print(cna.shape, therapy.shape, merged.shape)

(102, 23109) (102, 47) (102, 59)


In [117]:
clinical_n_cna = pd.concat([merged, cna], axis=1)

In [50]:
categories = list(range(len(columns)))
category_dict = dict(zip(columns, categories))

In [118]:
clinical_n_cna

,AGE_AT_DIAGNOSIS_GROUP,CALC_TIME_TO_METS_DX_DAYS,HORMONE_RECEPTOR_STATUS,HER2_STATUS_REPORTED,TRIPLE_NEG_STATUS,INFLAMMATORY_STATUS,HISPANIC,RACE_CATEGORIZED,T_STAGE_REPORTED,N_STAGE_REPORTED,...,RPL23AP82,SBF1,SCO2,SELO,SHANK3,SYCE3,TRABD,TUBGCP6,TYMP,ZBED4
MBC-MBCProject_1ps2hZH8-Tumor-SM-DL4OO,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
MBC-MBCProject_1qhlIasw-Tumor-SM-AXGNH,3.0,2175.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MBC-MBCProject_27uAugT4-Tumor-SM-DL45T,1.0,503.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
MBC-MBCProject_3LI1TNs3-Tumor-SM-DL43B,1.0,14.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MBC-MBCProject_3MUnT7Ty-Tumor-SM-CGLLE,5.0,3948.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
MBC-MBCProject_3jhES9fq-Tumor-SM-AXGIU,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MBC-MBCProject_43UpSwhz-Tumor-SM-DL4W8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
MBC-MBCProject_4DIpSBFV-Tumor-SM-CGM2Q,6.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
MBC-MBCProject_4MF1FlFQ-Tumor-SM-AZ5CU,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
MBC-MBCProject_4MF1FlFQ-Tumor-SM-CGM4M,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [6]:
enc = OneHotEncoder()

In [7]:
enc.fit(therapy)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [8]:
enc.n_values_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [9]:
enc.feature_indices_

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 25, 27, 29, 31,
       33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65,
       67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93])